## 0. Import libraries

In [ ]:
# for general use
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn import preprocessing

# for feature selection
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso

# for feature selection testing
from sklearn.neighbors import KNeighborsRegressor
import xgboost
from keras.models import Sequential
from keras.layers import Dense, Dropout

# for hyperparameter finetuning
import itertools
from bayes_opt import BayesianOptimization

# for model explaination
import sys
!{sys.executable} -m pip install shap
import shap

## 1. Read dataset

Remember to adjust the path 'dataset/dataset.shp' if needed

In [ ]:
# get the dataset
dataset = gpd.read_file('dataset/dataset.shp')

In [ ]:
# get only rows with AGB values and drop geometry
modeling_dataset = dataset.drop('geometry', 1)
modeling_dataset = pd.DataFrame(modeling_dataset.loc[dataset['AGB_t_ha']!=-9999.0])

In [ ]:
# separate modeling_dataset into features (X) and AGB (y)
X = modeling_dataset.drop('AGB_t_ha',1)
y = pd.DataFrame(modeling_dataset['AGB_t_ha'])

In [ ]:
# create a unique identifier for each forestry plot
# the default dataset contains a total of 73 forestry plots
plot_df = pd.DataFrame()
plot_df['plot_number']=np.argwhere(y.values == np.unique(y))[:,1]
# adjust index
plot_df.index=y.index

### 2. Define function: 7-folds cross-valalidation and normalization with RobustScaler

With the purpose of generating reliable results, we now built a function which performs data normalization with the RobustScaler and a 7-folds cross-validation by posing 2 constraints within the function. 
- Firstly, points belonging to the same original forestry plot can be included exclusively in the training or in the validation set; 
- Secondly, only the training set is used to fit the scaler, this allows validation data to remain unseen throughout the process. The entire dataset is finally normalized utilizing the trained scaler. 

This function is used before each step, that is feature selection, feature selection testing, and hyper-parameters fine-tuning.

In [ ]:
def cross_val(X,y,folds=7,random_state=42):
    
    ''' 
    cross_val function sets 2 contraints:
    - points belonging to the same plot (plot_df) end up in either the training or the validation set;
    - only the training set fits the scaler.
    it outputs training and validation set
    
               REQUIRES
               plot_df!
    
    plot_df: forestry plot unique identifier
    X: independent variables (Vegetation Indices, texture measures, etc.)
    y: target variable (AGB)
      
    '''
      
    kf = KFold(n_splits=folds)
    
    # interates over number of train_plots and val_plots for each fold
    for train_plots, val_plots in kf.split(np.unique(plot_df['plot_number'].unique())): 
        
        # goes to X, checks which entries have plot number in the train_plots and stacks the entries
        X_train =np.vstack([X[plot_df['plot_number']==plot].values for plot in train_plots])       
        # goes to X, checks which entries have plot number in the train_plots and stacks the entries
        X_val =np.vstack([X[plot_df['plot_number']==plot].values for plot in val_plots])
        
        # goes to y, checks which entries have plot number in the train_plots and stacks the entries
        y_train =pd.DataFrame(np.vstack([y[plot_df['plot_number']==plot].values for plot in train_plots]))
        y_train.columns=y.columns         
        # goes to y, checks which entries have plot number in the train_plots and stacks the entries
        y_val =pd.DataFrame(np.vstack([y[plot_df['plot_number']==plot].values for plot in val_plots]))
        y_train.columns=y.columns 
        
        # scale with RobustScaler
        scaler = preprocessing.RobustScaler().fit(X_train)
        X_train = pd.DataFrame(scaler.transform(X_train))
        X_train.columns=X.columns        
        X_val = pd.DataFrame(scaler.transform(X_val))
        X_val.columns=X.columns    
        yield (X_train,y_train),(X_val,y_val) 

## 3. Feature selection
**note:** the default dataset has 63 features. Adjust the code if you are working with different data.

In order to build a reliable model for the estimation of Above Ground Biomass (AGB), careful evaluation is required when deciding which and how many features to include in the predictive model. A total of 4 features selection methods are evaluated, including both supervised and unsupervised approaches. Other than the Mean Decrease in Impurity (MDI) and the Mean Decrease in Accuracy (MDA), this work explores the potential of L1 Regularization (LASSO); both of which are supervised methods - i.e. the target is known. Hence, an unsupervised method, Principal Component Analysis (PCA) is included, and its performance evaluated.

### 3.1. Feature selection w/ Random Forest Regressor (Mean Decrease in Impurity)
Impurity measures can be used for feature selection by evaluating the extent to which each feature contributes to decreasing the averaged impurity in each tree composing the forest, so as to calculate the MDI for each feature. The feature able to account for more variance decrease is going to be at the top of the ranking. Therefore, the MDI can be see as the total decrease in node impurity from splitting on the variable, averaged over all trees.

In [ ]:
# inititate an empty list
feature_imps_MDI=[]

for train,val in cross_val(X,y,folds=7,random_state=42):    
    
    # create a Random Forest model
    rf_model = RandomForestRegressor()   
    # fit on training data (0 is X_train, 1 is y_train)
    rf_model.fit(train[0], train[1].values.ravel())    
    # store feature importances of the folds in a list
    feature_imps_MDI.append(rf_model.feature_importances_)

# average of values over all folds
mean_values_MDI = np.mean(feature_imps_MDI,axis=0)

#save indexes (column order) for later
index_MDI = mean_values_MDI.argsort()

In [ ]:
# plot average feature importance of all folds
RF_feat_importance = pd.Series(mean_values_MDI, index=X.columns)
RF_feat_importance.nsmallest(63).plot(kind='barh', figsize=(5,15), color="#40d491ff")
plt.title('Features importance by Mean Decrease in Impurity (MDI)')
plt.xlabel('Mean Decrease in Impurity')
plt.tight_layout()
plt.show()

### 3.2. Feature selection w/ Random Forest Regressor (Mean Decrease in Accuracy)
Mean Decrease in Accuracy (MDA) performs a ranking of features based on the increasing mean error of a tree in the forest when the feature values are randomly shuffled though maintaining their distribution. For regression problems, the error refers to the Mean Squared Error (MSE). MDA is performed by measuring the impact of each feature on the model MSE by starting from a MSE
baseline.

In [ ]:
# initiate an empty list
feature_imps_MDA=[]

for train,val in cross_val(X,y,folds=7,random_state=42):
    
    # create a Random Forest model
    rf_model = RandomForestRegressor()
    # fit on training data (0 is X_train, 1 is y_train)
    rf_model.fit(train[0], train[1].values.ravel())
    # store feature importances of the folds in a list
    feature_imps_MDA.append(permutation_importance(rf_model, train[0], train[1]))


importances_mean_MDA = []
for index in range(len(feature_imps_MDA)):
    importances_mean_MDA.append(feature_imps_MDA[index]['importances_mean'])
    
# average values over all folds
mean_values_MDA = np.mean(importances_mean_MDA, axis=0)

# save indices (column order) for later
index_MDA = mean_values_MDA.argsort()

In [ ]:
# plot average feature imporance of all the folds
MDA_feat_importance = pd.Series(mean_values_MDA, index=X.columns)
MDA_feat_importance.nsmallest(63).plot(kind='barh', figsize=(5,15), color="#40d491ff")
plt.title('Features importance by Mean Decrease in Accuracy (MDA)')
plt.tight_layout()
plt.figure(figsize=(4,15))
plt.show()

### 3.3. Feature selection w/ Principal Component Analysis
How is it possible to do feature selection with PCA? We can use the loadings. What are the loeadings, you ask? Its the weight that each variable has for a components of PCA. Hence, a variable with higher loading means that it is more important for the explaining the variance of the data. Let's see what happens if we use **only the first principal component**.

In [ ]:
# inititate an empty list
Loadings_PC1=[]

for train,val in cross_val(X,y,folds=7,random_state=42):
    
    # create a pca model
    PCA_model = PCA(n_components=1).fit(train[0])
    Loadings_PC1.append(PCA_model.components_)

# average of values over all folds
mean_loadings_PCA = np.mean(Loadings_PC1,axis=0)

In [ ]:
# plot loading values for the first principal component
plt.figure(figsize=(4,15))
index = np.argsort(np.abs(mean_loadings_PCA[0]))
plt.barh(X.columns[index],np.sort(np.abs(mean_loadings_PCA[0])),color="#40d491ff")
plt.title('Loading Values for the first Principal Component')
plt.show()

##### ...but what about the other components? 
Thats a good question! In fact, the first component only accounts for a percentage of the total variation of the data. Let us check the loadings of our variables in the **first and second components**. 

In [ ]:
# inititate an empty list
Loadings_PC1=[]
Loadings_PC2=[]

for train,val in cross_val(X,y,folds=7,random_state=42):
    
    # create a PCA model
    PCA_model = PCA(n_components=2).fit(train[0])
    Loadings_PC1.append( PCA_model.components_[0])
    Loadings_PC2.append( PCA_model.components_[1])


# average of values over all folds
mean_loadings_PC1 = np.mean(Loadings_PC1,axis=0)
mean_loadings_PC2 = np.mean(Loadings_PC2,axis=0)

In [ ]:
# plot loading values for the first and second principal components
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8,12))
index = np.argsort(np.abs(mean_loadings_PC1))
# generate first plot
ax1.barh(X.columns[index],np.abs(mean_loadings_PC1)[index],color="#40d491ff")
ax1.set_title(f'First Principal Component {np.round(100*PCA_model.explained_variance_ratio_[0],1)}%')
# generate second plot
ax2.barh(X.columns[index],np.abs(mean_loadings_PC2)[index],color="#40d491ff")
ax2.set_title(f'Second Principal Component {np.round(100*PCA_model.explained_variance_ratio_[1],1)}%')

plt.tight_layout()

##### That's cool and all but, how do we do the feature selection then?
We are going to do a weighted sum over a sufficent number of principal components. We sum the loading and weight it with the explained variance ratio of each componet.

##### But what is a sufficient number of components?
Thats a good question. Eventually the total explained variance starts to converge as we add more components. We are now going to test that.

In [ ]:
# inititate an empty list
Loadings=[]

for train,val in cross_val(X,y,folds=7,random_state=42):
    
    # create a PCA model
    # you can experiment with different number of components
    PCA_model = PCA(n_components=25).fit(train[0])
    Loadings.append(PCA_model.components_[0])

# average of values over all folds
mean_loadings = np.mean(Loadings,axis=0)

In [ ]:
# plot the cumulative explained variance
plt.bar(np.arange(25)[:11],np.cumsum(PCA_model.explained_variance_ratio_)[:11],color="#40d491ff",alpha=0.9)
plt.bar(np.arange(25)[11:],np.cumsum(PCA_model.explained_variance_ratio_)[11:],alpha=0.3,color="#40d491ff")
plt.hlines(0.95,xmin=0, xmax=25, linestyle='--', color='#ff7f4dff')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.show()

Let's know check whether the order of the top ranked features changes when using 5, 10 and 20 components. We first need to create a function which calculates the weight of the components.

In [ ]:
def molisse_method(n_components, X):
    
    '''
    molisse_method finds the weight (i.e. dataset variance explained) of each component to after perform feature selection
    
    n_components:number of principal components
    X: dataset variables
    
    '''
    pca_model = PCA(n_components=n_components).fit(X)
    weight = np.zeros(shape=(X.shape[1]))
    for component in range(n_components):
        weight += np.abs(pca_model.components_[component])*pca_model.explained_variance_[component]
    return weight

In [ ]:
# inititate empty list
Loadings_5 =[]
Loadings_10 =[]
Loadings_20 =[]
weights_5 = []
weights_10 = []
weights_20 = []

for train,val in cross_val(X,y,folds=7,random_state=42):
    
    weights_5.append(molisse_method(5,train[0]))
    weights_10.append(molisse_method(10,train[0]))
    weights_20.append(molisse_method(20,train[0]))
    
    PCA_model_5 = PCA(n_components=5).fit(train[0])
    PCA_model_10 = PCA(n_components=10).fit(train[0])
    PCA_model_20 = PCA(n_components=20).fit(train[0])
    
    Loadings_5.append(PCA_model_5.components_[0])
    Loadings_10.append(PCA_model_10.components_[0])
    Loadings_20.append(PCA_model_20.components_[0])

# mean of values over all folds
mean_weights_5 = np.mean(weights_5, axis=0)
mean_weights_10 = np.mean(weights_10, axis=0)
mean_weights_20 = np.mean(weights_20, axis=0)

mean_loadings_5 = np.mean(Loadings_5,axis=0)
mean_loadings_10 = np.mean(Loadings_10,axis=0)
mean_loadings_20 = np.mean(Loadings_20,axis=0)

In [ ]:
# plot weighted loading values for the 5, 10 and 20 components
fig, (ax1, ax2,ax3) = plt.subplots(1, 3, figsize=(12,12))


index_weights_5 = np.argsort(mean_weights_5)
ax1.barh(X.columns[index_weights_5],mean_weights_5[index_weights_5],color="#40d491ff")
ax1.set_title(f'5 Principal Components')

index_weights_10 = np.argsort(mean_weights_10)
ax2.barh(X.columns[index_weights_10],mean_weights_10[index_weights_10],color="#40d491ff")
ax2.set_title(f'10 Principal Components')

index_weights_20 = np.argsort(mean_weights_20)
ax3.barh(X.columns[index_weights_20],mean_weights_10[index_weights_20],color="#40d491ff")
ax3.set_title(f'20 Principal Components')

plt.tight_layout()

# we pick the result from 10 components
# save 10 components indices (column order) for later
index_PCA_10 = index_weights_10

### 3.4. Feature selection w/ L1 regularization (LASSO)
Regularization algorithms aim to minimize the residual sum of squares of the model by using the Ordinary Least Square (OLS); this is achieved by shrinking the estimated regression coefficients approaching to zero. The Least Absolute Shrinkage and Selection Operator (LASSO) was proposed by Tibshirani (1996); LASSO regression is an L1 Regularization technique, which minimizes the absolute sum of the coefficients; while L2 regularization minimizes the squared sum of the coefficients.

Feature selection with LASSO is performed by multiplying every coefficient by a constant, that is the regularization parameter alpha (α). Alpha controls the strength of penalty, meaning that for large values of alpha, a large number of coefficients is forced to zero which corresponds to the exclusion of those features from the model; on the other hand, when alpha is equal to zero, we have the classic OLS.

In [ ]:
# this cell is going to give some warnings asking to increase the number of iterations. feel free to adapt it.
# initialize empty list
count = []

for train,val in cross_val(X,y,folds=7,random_state=42):
    count.append([np.sum(Lasso(alpha=cost,max_iter=20000).fit(train[0],train[1]).coef_!=0).astype(int) for cost in np.linspace(0,10,1000)[::-1]])

# average of values over all folds
mean_count = np.mean(count, axis=0, dtype=np.int)

In [ ]:
# plot number of selected features vs regularization parameter values
plt.plot(np.linspace(0,10,1000)[::-1],mean_count,color="#40d491ff")
plt.xlabel('Regularization parameter')
plt.ylabel('Number of features selected')
plt.show()

In [ ]:
binary_selection_f = []
weights_selection_f = []

binary_selection=np.zeros(shape=(63))
weights_selection = np.zeros(shape=(63))

    
for train,val in cross_val(X,y,folds=7,random_state=42):
    
    for cost in np.linspace(0,10,1000)[::-1]:
        selector = Lasso(alpha=cost,max_iter=5000).fit(train[0],train[1])
        binary_selection += (selector.coef_!=0).astype(int)
        weights_selection += np.abs(selector.coef_)
        
    binary_selection_f.append(binary_selection)
    weights_selection_f.append(weights_selection)


mean_binary_selection=np.mean(binary_selection_f, axis=0)
mean_weights_selection=np.mean(weights_selection_f, axis = 0)

In [ ]:
# plot feature importance

# this plot adds up to 7000 because a range from 0 to 10, with a total of 1000 values, was chose for alpha;
# this was looped over using 7-fold crossvalidation -> 7000
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,15))
index_lasso = np.argsort(mean_binary_selection)
ax1.barh(X.columns[index_lasso],mean_binary_selection[index_lasso], color="#40d491ff")
ax1.set_title('L1 Selection Ranking')
ax1.set_xlabel('Frequency of selection')

ax2.barh(X.columns[index_lasso],mean_weights_selection[index_lasso], color="#40d491ff")
ax2.set_title('Total Sum of weights')

plt.tight_layout()
plt.show()

## 4. Feature selection testing & hyper-parameters optimization




In the following cells, the previously explained feature selection methods are tested and evaluated using several non-parametric Machine Learning (ML) algorithms: K-Nearest Neighbour (kNN), Random Forest (RF), Extreme Gradient Boosting (XGB) and, lastly, 3 Artificial Neural Networks (ANN).

In [ ]:
# create a list with all the feature selection rankings to be tested
index_list = [index_MDI,index_MDA,index_PCA_10,index_lasso]
# create a list with feature selection methods names
names = ['MDI','MDA','PCA','LASSO']

### 4.1. k-Nearest Neighbour

### 4.1.1. k-Nearest Neighbour: feature selection testing

In [ ]:
fold_results=[]
for train, val in cross_val(X,y,folds=7,random_state=42):
    results=[]
    for index in index_list:
        selected_features = []
        method_results=[]
        for feature in train[0].columns[index[::-1]]:
            selected_features.append(feature) 
            
            model = KNeighborsRegressor().fit(train[0][selected_features], train[1].values.flatten())
            KNN_preds = model.predict(val[0][selected_features])
            rmse = np.sqrt(np.mean(np.square(KNN_preds-val[1].values.flatten())))
            
            method_results.append(rmse)
        results.append(method_results)
    fold_results.append(results)


results = np.mean(fold_results,axis=0)
error = np.std(fold_results,axis=0)

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(13,5))

winner = np.argwhere(results == np.min(results)).flatten()[0]

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax1.plot(results[i],label=names[i], color=colors[i])
ax1.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax1.set_title(f'Regressor: kNN baseline \nMinimum RMSE: {np.round(np.min(results),2)} t/ha \n Method: {names[winner]}')
ax1.set_ylabel('RMSE (t/ha)')
ax1.set_xlabel('Number of features')

ax1.legend()

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax2.plot(results[i],label=names[i], color=colors[i])
ax2.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='Min RMSE')
ax2.set_title(f'Regressor: kNN baseline \nMinimum RMSE: {np.round(np.min(results),2)} t/ha \n Method: {names[winner]}')
ax2.set_xlim([19,26])
ax2.set_ylim([43.5,46])
ax2.set_ylabel('RMSE (t/ha)')
ax2.set_xlabel('Number of features')

ax2.legend()
fig.show()

 **IMPORTANT :**   
Create a list of the best performing features. Such that, if the best perfoming k-NN model was achieved by using the first 15 features selected by the MDA feature selection method, create a list with such fueatures.

i.e. add a cell such that

kNN_X = X [ [ ' feature1 ' , ' feature2 ' , ' feature3 ' , ... , ' featureN ' ] ]


### 4.1.2. k-Nearest Neighbour: hyper-parameters optimization (Grid Search)

In [ ]:
# note: kNN_X is the list of features you created in the previous step
# you need to define kNN_X

# list Hyperparameters that we want to tune.
n_neighbors = (range(1,51))
# convert to dictionary
kNN_hyperparameters = dict(n_neighbors=n_neighbors)
# scoring matrix
scoring = ["neg_mean_squared_error"]

# initialize empty list
neighbor_loss = []

for n_neighbors in np.arange(1,50):
    rmse=[]
    for train, val in cross_val(kNN_X,y,folds=7):
        model = KNeighborsRegressor(n_neighbors=n_neighbors)
        model.fit(train[0],train[1].values.flatten())
        preds = model.predict(val[0])
        rmse.append(np.sqrt(np.mean(np.square(preds-val[1].values.flatten()))))
          
    neighbor_loss.append(rmse)

In [ ]:
plt.plot(range(1,50), np.mean(neighbor_loss, axis=1))
plt.hlines(np.min(np.mean(neighbor_loss, axis=1)),xmin=0,xmax=63,linestyle='--',color='#25521988',label='Min RMSE')
plt.title(f'kNN w/ GridSearch \nMinimum RMSE: {np.round(np.min(np.mean(neighbor_loss, axis=1)), 2)} t/ha, \nNumber of neighbours: 11')
plt.xlabel("Number of neighbours")
plt.ylabel("RMSE (t/ha)")
plt.xlim([0,50])
plt.ylim([42,50])
plt.legend(loc='upper right')
plt.show()

### 4.2. Random Forest
### 4.2.1. Random Forest: feature selection testing

In [ ]:
fold_results=[]
for train, val in cross_val(X,y,folds=7,random_state=42):
    results=[]
    for index in index_list:
        selected_features = []
        method_results=[]
        for feature in train[0].columns[index[::-1]]:
            selected_features.append(feature)

            model  = RandomForestRegressor(random_state=42).fit(train[0][selected_features], train[1].values.flatten())
            RF_preds = model.predict(val[0][selected_features])
            rmse = np.sqrt(np.mean(np.square(RF_preds-val[1].values.flatten())))

            method_results.append(rmse)
        results.append(method_results)
    fold_results.append(results)
    
results = np.mean(fold_results,axis=0)
error = np.std(fold_results,axis=0)

In [ ]:
# plot the feature selection testing w/ RF
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(13,5))

winner = np.argwhere(results == np.min(results)).flatten()[0]

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    #ax1.fill_between(np.arange(63), results[i]-error[i], results[i]+error[i],alpha=0.15, color=colors[i])
    ax1.plot(results[i],label=names[i], color=colors[i])
ax1.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax1.set_title(f'Random Forest (baseline) \nMinimum RMSE: {np.round(np.min(results),2)}t/ha \n Method: {names[winner]}')
ax1.set_ylabel('RMSE (t/ha)')
ax1.set_xlabel('Number of features')
#plt.ylabel("Root Mean Squared Error")
#ax1.text(63, np.round(np.min(results)), 'Minimum')

ax1.legend(loc='upper right')

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    #ax2.fill_between(np.arange(63), results[i]-error[i], results[i]+error[i],alpha=0.15, color=colors[i])
    ax2.plot(results[i],label=names[i], color=colors[i])
ax2.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax2.set_title(f'Random Forest (baseline) \nMinimum RMSE: {np.round(np.min(results),2)}t/ha \n Method: {names[winner]}')
ax2.set_ylabel('RMSE (t/ha)')
ax2.set_xlabel('Number of features')
ax2.set_xlim([2,8])
ax2.set_ylim([41,60])
#plt.xlabel("Number of included features")

#ax2.text(2, np.round(np.min(results)), 'Minimum')
ax2.legend(loc='upper right')
fig.show()

 **IMPORTANT :**   
Create a list of the best performing features. Such that, if the best perfoming RF model was achieved by using the first 15 features selected by the MDA feature selection method, create a list with such fueatures.

i.e. add a cell such that

rf_X = X [ [ ' feature1 ' , ' feature2 ' , ' feature3 ' , ... , ' featureN ' ] ]


### 4.2.2. Random Forest: hyper-parameter optimization (Grid Search)

In [ ]:
# note: rf_X is the list of features you created in the previous step
# you need to define rf_X

grid_best_loss = 1000
grid_parameter_loss=[]
# define a list with hyperparameters to be tested
for n_estimators,max_depth,min_samples_split, min_samples_leaf in list(itertools.product(np.arange(30,1000),
                                                                                        np.arange(10,100),
                                                                                        np.arange(2,5),
                                                                                       np.arange(1,2))):
    rmse=[]
    for train, val in cross_val(rf_X,y,folds=7,random_state=42):
        model = RandomForestRegressor(random_state=42, n_estimators=n_estimators,
                                      max_depth = max_depth,
                                      min_samples_split=min_samples_split,
                                      min_samples_leaf=min_samples_leaf)
        model.fit(train[0],train[1].values.flatten())
        preds = model.predict(val[0])
        rmse.append(np.sqrt(np.mean(np.square(preds-val[1].values.flatten()))))
    
    if np.mean(rmse) < grid_best_loss:
        grid_best_parameters = [n_estimators, max_depth, min_samples_split, min_samples_leaf]
        grid_best_loss = np.mean(rmse)
    grid_parameter_loss.append(np.mean(rmse))

In [ ]:
# print the results of grid search
print ("Random Forest with GridSearch","\n_","\nRMSE:", grid_best_loss ,"\n_","\nn_estimators:", grid_best_parameters[0], 
       "\nmax_depth:", grid_best_parameters[1],"\nmin_samples_split:", grid_best_parameters[2], 
       "\nmin_samples_leaf:", grid_best_parameters[3])

### 4.2.3. Random Forest: hyper-parameter optimization (Random Search)

In [ ]:
# define a dictionary with hyperparameters to be tested
random_hyperparameters = dict(n_estimators=np.arange(30,1000,10),
                        max_depth=np.arange(10,100,10),
                        min_samples_split = np.arange(2,5,10),
                        min_samples_leaf=np.arange(1,2,4),
                       )

In [ ]:
# note: rf_X is the list of features you created in the previous step
# you need to define rf_X

n_iterations = 100
best_loss = 1000
for _ in range(n_iterations):
    n_estimators = np.random.choice(random_hyperparameters['n_estimators'],replace=True)
    max_depth=np.random.choice(random_hyperparameters['max_depth'],replace=True)
    min_samples_split = np.random.choice(random_hyperparameters['min_samples_split'],replace=True)
    min_samples_leaf=np.random.choice(random_hyperparameters['min_samples_leaf'],replace=True)
    
    rmse = []
    for train, val in cross_val(rf_X,y,folds=7,random_state=42):
        model = RandomForestRegressor(random_state=42, n_estimators=n_estimators,
                                      max_depth = max_depth,
                                      min_samples_split=min_samples_split,
                                      min_samples_leaf=min_samples_leaf)
        model.fit(train[0],train[1].values.flatten())
        preds = model.predict(val[0])
        rmse.append(np.sqrt(np.mean(np.square(preds-val[1].values.flatten()))))
        
    if np.mean(rmse) < best_loss:
        best_parameters = [n_estimators, max_depth, min_samples_split, min_samples_leaf]
        best_loss = np.mean(rmse)

In [ ]:
# print the results of randoom search
print ("Random Forest with RandomSearch","\n_","\nRMSE:", best_loss ,"\n_","\nn_estimators:", best_parameters[0], 
       "\nmax_depth:", best_parameters[1],"\nmin_samples_split:", best_parameters[2], 
       "\nmin_samples_leaf:", best_parameters[3])

### 4.2.4. Random Forest: hyper-parameter optimization (Bayesian Search)

In [ ]:
# define a dictionary with hyperparameters to be tested
bayesian_hyperparameters = {"n_estimators": (30,1000),
                        "max_depth": (1,100),
                        "min_samples_split": (2,5),
                        "min_samples_leaf": (1,2)}

In [ ]:
# note: rf_X is the list of features you created in the previous step
# you need to define rf_X

# define fuction to be maximised (negative RMSE)
def BayesianForest(n_estimators,max_depth, min_samples_split,min_samples_leaf):
        
    rmse = []
    for train, val in cross_val(rf_X,y,folds=7,random_state=42):
        model = RandomForestRegressor(random_state=42, n_estimators=int(n_estimators),
                                    max_depth=int(max_depth),
                                    min_samples_split=int(min_samples_split),
                                    min_samples_leaf=int(min_samples_leaf))
    
        model.fit(train[0],train[1].values.flatten())
        preds = model.predict(val[0])
        rmse.append(np.sqrt(np.mean(np.square(preds-val[1].values.flatten()))))

    return -np.mean(rmse)

In [ ]:
# define optimizer
optimizer = BayesianOptimization(
    f=BayesianForest,
    pbounds=bayesian_hyperparameters,
    random_state=42,)

In [ ]:
# run optimizer !
optimizer.maximize(init_points=200, n_iter=1000)

In [ ]:
# print optimizer results
print(optimizer.max)

### 4.3. Extreme Gradent Boosting

### 4.3.1. Extreme Gradent Boosting: feature selection testing

In [ ]:
fold_results=[]
for train, val in cross_val(X,y,folds=7,random_state=42):
    results=[]
    for index in index_list:
        selected_features = []
        method_results=[]
        for feature in train[0].columns[index[::-1]]:
            selected_features.append(feature)
            
            model = xgboost.XGBRegressor(random_state=42)
            model.fit(train[0][selected_features], train[1].values.flatten(),
                      eval_metric='rmse', verbose=True)
            
            preds = model.predict(val[0][selected_features])
            rmse = np.sqrt(np.mean(np.square(preds-val[1].values.flatten())))

            method_results.append(rmse)
        results.append(method_results)
    fold_results.append(results)
    
results = np.mean(fold_results,axis=0)


In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(13,5))

winner = np.argwhere(results == np.min(results)).flatten()[0]

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax1.plot(results[i],label=names[i], color=colors[i])
ax1.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax1.set_title(f'Regressor: XGB baseline \nMinimum RMSE: {np.round(np.min(results),2)} t/ha \n Method: {names[winner]}')
ax1.set_ylabel('RMSE (t/ha)')
ax1.set_xlabel('Number of features')


ax1.legend(loc='upper right')

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax2.plot(results[i],label=names[i], color=colors[i])
ax2.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='Min RMSE')
ax2.set_title(f'Regressor: XGB baseline \nMinimum RMSE: {np.round(np.min(results),2)} t/ha \n Method: {names[winner]}')
ax2.set_xlim([12,30])
ax2.set_ylim([45,49])
ax2.set_ylabel('RMSE (t/ha)')
ax2.set_xlabel('Number of features')

ax2.legend(loc='upper right')
fig.show()

 **IMPORTANT :**   
Create a list of the best performing features. Such that, if the best perfoming XGB model was achieved by using the first 15 features selected by the MDA feature selection method, create a list with such fueatures.

i.e. add a cell such that

xgb_X = X [ [ ' feature1 ' , ' feature2 ' , ' feature3 ' , ... , ' featureN ' ] ]

### 4.3.2. Extreme Gradent Boosting (XGB): hyper-parameter optimization (Random Search)

In [ ]:
# define a dictionary with hyperparameters to be tested
random_hyperparameters = dict(alpha = np.arange(0,0.9), gamma = np.arange(0,0.9),
                              learning_rate = np.arange(0,0.9), n_estimators = np.arange(30,1000),
                              max_depth = np.arange(1,8), subsample = np.arange(0,0.9))

In [ ]:
# note: xgb_X is the list of features you created in the previous step
# you need to define xgb_X

n_iterations = 200
best_loss = 1000
for _ in range(n_iterations):
    
    alpha = np.random.choice(random_hyperparameters['alpha'],replace=True)
    gamma=np.random.choice(random_hyperparameters['gamma'],replace=True)
    learning_rate = np.random.choice(random_hyperparameters['learning_rate'],replace=True)
    n_estimators = np.random.choice(random_hyperparameters['n_estimators'],replace=True)
    max_depth=np.random.choice(random_hyperparameters['max_depth'],replace=True)
    subsample = np.random.choice(random_hyperparameters['subsample'],replace=True)
    
    rmse = []
    for train, val in cross_val(xgb_X,y,folds=7,random_state=42):
        
        model_random = xgboost.XGBRegressor(alpha = int(alpha), max_depth = int(max_depth),
              gamma = gamma, n_estimators= int(n_estimators), learning_rate=learning_rate, 
                                     subsample = subsample, eval_metric = 'rmse', random_state=42)        
        model_random.fit(train[0], train[1].values.flatten(), eval_metric='rmse', 
                  verbose=True)            
        preds = model_random.predict(val[0])
        rmse.append(np.sqrt(np.mean(np.square(preds-val[1].values.flatten()))))
        
        
    if np.mean(rmse) < best_loss:
        best_parameters = [alpha, gamma, learning_rate, n_estimators, max_depth, subsample ]
        best_loss = np.mean(rmse)

In [ ]:
# print best hyperparameters results
print ("XGradientBoosting with RandomSearch","\n_","\nRMSE:", best_loss ,"\n_","\nalpha:", best_parameters[0], 
       "\ngamma:", best_parameters[1],"\nlearning_rate:", best_parameters[2], 
       "\nn_estimators:", best_parameters[3], "\nmax_depth:", best_parameters[4], "\nsubsample", best_parameters[5] )

### 4.3.3. Extreme Gradient Boosting (XGB): hyper-parameter optimization (Bayesian Search)

In [ ]:
# define a dictionary with hyperparameters to be tested
hy_params = {"alpha": (0,0.9), 'subsample': (0,1),'gamma': (0, 0.9),'learning_rate':(0,0.9),'n_estimators':(30,1000), "max_depth": (1,8)}

In [ ]:
# note: xgb_X is the list of features you created in the previous step
# you need to define xgb_X

# define function to be maximized (negative RMSE)
def BayesianForest(max_depth, gamma, n_estimators ,learning_rate, alpha, subsample):
    rmse = []
    for train, val in cross_val(xgb_X,y,folds=7):
        
        model = xgboost.XGBRegressor(alpha = int(alpha), max_depth = int(max_depth),
              gamma = gamma,
              n_estimators= int(n_estimators),
              learning_rate=learning_rate, subsample = subsample,
              eval_metric = 'rmse', random_state=42)
        
        model.fit(train[0], train[1].values.flatten(), eval_metric='rmse', 
                  verbose=True)
            
        preds = model.predict(val[0])
        rmse.append(np.sqrt(np.mean(np.square(preds-val[1].values.flatten()))))
    
    return -np.mean(rmse)

In [ ]:
# define optimizer
optimizer = BayesianOptimization(
    f=BayesianForest, pbounds=hy_params
    )

In [ ]:
# run it
optimizer.maximize(init_points=150, n_iter=600)

In [ ]:
# print hyperparameters that maximise the -RMSE
print(optimizer.max)

### 4.4. Linear Neural Network (LNN): feature selection testing

In [ ]:
fold_results=[]
for train, val in cross_val(X,y,folds=7,random_state=42):
    results=[]
    for index in index_list:
        selected_features = []
        method_results=[]
        for feature in train[0].columns[index[::-1]]:
            selected_features.append(feature)
            
            linear = Sequential()
            linear.add(Dense(units=1,activation='linear',  dtype='float64'))
            linear.compile(optimizer='adam', loss='mse')
            
            linear.fit(train[0][selected_features], train[1].values.flatten(), epochs = 500, verbose=0)
            linear_preds = linear.predict(val[0][selected_features])
            rmse = np.sqrt(np.mean(np.square(linear_preds-val[1].values.flatten())))

            method_results.append(rmse)
        results.append(method_results)
    fold_results.append(results)
    
results = np.mean(fold_results,axis=0)

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(13,5))

winner = np.argwhere(results == np.min(results)).flatten()[0]

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax1.plot(results[i],label=names[i], color=colors[i])
ax1.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax1.set_title(f'ANN (linear) \nMinimum RMSE: {np.round(np.min(results),2)}t/ha \n Method: {names[winner]}')
ax1.set_ylabel('RMSE (t/ha)')
ax1.set_xlabel('Number of features')

ax1.legend(loc='upper right')

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax2.plot(results[i],label=names[i], color=colors[i])
ax2.hlines(np.min(results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax2.set_title(f'ANN (linear) \nMinimum RMSE: {np.round(np.min(results),2)}t/ha \n Method: {names[winner]}')
ax2.set_ylabel('RMSE (t/ha)')
ax2.set_xlabel('Number of features')
ax2.set_xlim([30,50])
ax2.set_ylim([75,80])

ax2.legend(loc='upper right')
fig.show()

### 4.5. Shallow Neural Network (SNN): feature selection testing

In [ ]:
simple_fold_results=[]
for train, val in cross_val(X,y,folds=7,random_state=42):
    simple_results=[]
    for index in index_list:
        selected_features = []
        simple_method_results=[]
        for feature in train[0].columns[index[::-1]]:
            selected_features.append(feature)
                     
            simple = Sequential()
            simple.add(Dense(units=8,activation='relu',  dtype='float64'))
            simple.add(Dense(units=4,activation='relu',  dtype='float64'))
            simple.add(Dense(units=1,activation='linear',  dtype='float64'))
            
            simple.compile(loss='mse',optimizer='adam')
            
            simple.fit(train[0][selected_features],train[1].values.flatten(),epochs=500, verbose=0)
            
            simple_preds = simple.predict(val[0][selected_features])
            simple_rmse = np.sqrt(np.mean(np.square(simple_preds-val[1].values.flatten())))

            
            simple_method_results.append(simple_rmse)
        simple_results.append(simple_method_results)
    simple_fold_results.append(simple_results)
    
simple_results = np.mean(simple_fold_results,axis=0)

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(13,5))

winner = np.argwhere(simple_results == np.min(simple_results)).flatten()[0]

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax1.plot(simple_results[i],label=names[i], color=colors[i])
ax1.hlines(np.min(simple_results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax1.set_title(f'ANN (simple) \nMinimum RMSE: {np.round(np.min(simple_results),2)}t/ha \n Method: {names[winner]}')
ax1.set_ylabel('RMSE (t/ha)')
ax1.set_xlabel('Number of features')


ax1.legend(loc='upper right')

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax2.plot(simple_results[i],label=names[i], color=colors[i])
ax2.hlines(np.min(simple_results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax2.set_title(f'ANN (simple) \nMinimum RMSE: {np.round(np.min(simple_results),2)}t/ha \n Method: {names[winner]}')
ax2.set_ylabel('RMSE (t/ha)')
ax2.set_xlabel('Number of features')
ax2.set_xlim([0,30])
ax2.set_ylim([41,50])

ax2.legend(loc='upper right')
fig.show()

### 4.6. Deep Neural Network (DNN): feature selection testing

In [ ]:
plex_fold_results=[]
for train, val in cross_val(X,y,folds=7,random_state=42):
    plex_results=[]
    for index in index_list:
        selected_features = []
        plex_method_results=[]
        for feature in train[0].columns[index[::-1]]:
            selected_features.append(feature)
            plex = Sequential()
            plex.add(Dense(units=128,activation='relu',  dtype='float64'))
            plex.add(Dropout(0.3))
            plex.add(Dense(units=64,activation='relu',  dtype='float64'))
            plex.add(Dense(units=32,activation='relu',  dtype='float64'))
            plex.add(Dropout(0.1))
            plex.add(Dense(units=16,activation='relu',  dtype='float64'))
            plex.add(Dense(units=8,activation='relu',  dtype='float64'))
            plex.add(Dense(units=1,activation='linear',  dtype='float64'))

            plex.compile(loss='mse', optimizer='adam') 

            plex.fit(train[0][selected_features],train[1].values.flatten(),epochs=500, verbose=0)
            
            plex_preds = plex.predict(val[0][selected_features])
            plex_rmse = np.sqrt(np.mean(np.square(plex_preds-val[1].values.flatten())))
            
            plex_method_results.append(plex_rmse)
        plex_results.append(plex_method_results)
    plex_fold_results.append(plex_results)
    
plex_results = np.mean(plex_fold_results,axis=0)

In [ ]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(13,5))

winner = np.argwhere(plex_results == np.min(plex_results)).flatten()[0]

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax1.plot(plex_results[i],label=names[i], color=colors[i])
ax1.hlines(np.min(plex_results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax1.set_title(f'ANN (complex) \nMinimum RMSE: {np.round(np.min(plex_results),2)}t/ha \n Method: {names[winner]}')
ax1.set_ylabel('RMSE (t/ha)')
ax1.set_xlabel('Number of features')

ax1.legend(loc='upper right')

for i in range(4):
    colors = ["#40d491ff","#ffe24dff","#ff7f4dff","#397abdff"]
    ax2.plot(plex_results[i],label=names[i], color=colors[i])
ax2.hlines(np.min(plex_results),xmin=0,xmax=63,linestyle='--',color='#25521988',label='min RMSE')
ax2.set_title(f'ANN (complex) \nMinimum RMSE: {np.round(np.min(plex_results),2)}t/ha \n Method: {names[winner]}')
ax2.set_ylabel('RMSE (t/ha)')
ax2.set_xlabel('Number of features')
ax2.set_xlim([0,10])
ax2.set_ylim([41,48])

ax2.legend(loc='upper right')
fig.show()

## 5. Goodness of fit

The following cells address goodness of fit of the Extreme Gradent Boosting (XGB) predictions. Add new cells to assess the goodness of fit of the other models predictions.

In [ ]:
# for each fold, put predictions and true measures of the XGB into arrays 
true_array = np.asarray(true, dtype=object)
preds_array = np.asarray(pred, dtype=object)

In [ ]:
# save prediction and true values to folder
np.savetxt("predictions/true_fold_1.csv", true_array[0])
np.savetxt("predictions/true_fold_2.csv", true_array[1])
np.savetxt("predictions/true_fold_3.csv", true_array[2])
np.savetxt("predictions/true_fold_4.csv", true_array[3])
np.savetxt("predictions/true_fold_5.csv", true_array[4])
np.savetxt("predictions/true_fold_6.csv", true_array[5])
np.savetxt("predictions/true_fold_7.csv", true_array[6])

np.savetxt("predictions/pred_fold_1.csv", preds_array[0])
np.savetxt("predictions/pred_fold_2.csv", preds_array[1])
np.savetxt("predictions/pred_fold_3.csv", preds_array[2])
np.savetxt("predictions/pred_fold_4.csv", preds_array[3])
np.savetxt("predictions/pred_fold_5.csv", preds_array[4])
np.savetxt("predictions/pred_fold_6.csv", preds_array[5])
np.savetxt("predictions/pred_fold_7.csv", preds_array[6])

In [ ]:
# read the predictions and true values
true_fold_1 = pd.read_csv("predictions/true_fold_1.csv")
pred_fold_1 = pd.read_csv("predictions/pred_fold_1.csv")
true_fold_2 = pd.read_csv("predictions/true_fold_2.csv")
pred_fold_2 = pd.read_csv("predictions/pred_fold_2.csv")
true_fold_3 = pd.read_csv("predictions/true_fold_3.csv")
pred_fold_3 = pd.read_csv("predictions/pred_fold_3.csv")
true_fold_4 = pd.read_csv("predictions/true_fold_4.csv")
pred_fold_4 = pd.read_csv("predictions/pred_fold_4.csv")
true_fold_5 = pd.read_csv("predictions/true_fold_5.csv")
pred_fold_5 = pd.read_csv("predictions/pred_fold_5.csv")
true_fold_6 = pd.read_csv("predictions/true_fold_6.csv")
pred_fold_6 = pd.read_csv("predictions/pred_fold_6.csv")
true_fold_7 = pd.read_csv("predictions/true_fold_7.csv")
pred_fold_7 = pd.read_csv("predictions/pred_fold_7.csv")

In [ ]:
# plot measured and predicted values
plt.scatter(true_fold_1, pred_fold_1, label='Fold 1', color= '#FD3838')
plt.scatter(true_fold_2, pred_fold_2, label='Fold 2', color= '#F98B2B')
plt.scatter(true_fold_3, pred_fold_3, label='Fold 3', color= '#F3E438')
plt.scatter(true_fold_4, pred_fold_4, label='Fold 4', color= '#84F936')
plt.scatter(true_fold_5, pred_fold_5, label='Fold 5', color='#4BF1D8')
plt.scatter(true_fold_6, pred_fold_6, label='Fold 6', color='#5D79F4')
plt.scatter(true_fold_7, pred_fold_7, label='Fold 7', color='#FF8BD6')


plt.axline([-1, -1], [1, 1], linestyle='--', color='#25521988', label='x = y')

plt.xlabel('Measured AGB (t/ha)')
plt.ylabel('Predicted AGB (t/ha)')
plt.axis('square')
plt.title('XGB (baseline) \nGoodness of fit on unseen data \nMeasured vs Predicted')
plt.legend(bbox_to_anchor=(1, 1.03))
plt.show()

## 6. Model Explanation with SHAP
Ideally, a Machine Learning (ML) model should be highly accurate and simple to interpret. By ”simple to interpret” is intended the ability to expose its performance in an understandable and intuitive way. Unfortunately, the more the model complexity increases, the harder it is to understand how certain values were predicted and which features had contributed more to those predictions. When working with a simple model, the impact that a feature has on the model output is easily interpretable by looking at its weights; whereas, complex models such as ensemble methods or deep networks are not as easy to understand; in this scenario, a model explainer can help interpreting the model results.

In the following cells, the SHapley Additive exPlanations (SHAP) package was used as model explainer. SHAP was created by Lundberg et al. (2020) and, according to its author, its implementation allows for appropriate user trust, provides insights for model improvement, and supports the understanding of the problem being modelled.

In [ ]:
shap.initjs()

In [ ]:
# initialize a model, or use one of the previously created models
# in this example we initialize an XGB model
model_xgb = xgboost.XGBRegressor(random_state=42)    
model_xgb.fit(xgb_X, y.values.flatten())

In [ ]:
explainer_xgb = shap.TreeExplainer(model_xgb)
shap_values_xgb = explainer_xgb.shap_values(xgb_X)

In [ ]:
# summarize plot which shoes the effects of all the features on model output
shap.summary_plot(shap_values_xgb, xgb_X)